In [2]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_score, recall_score
# from SimpleMKL import SimpleMKL


In [3]:
# Generate a custom dataset
X, y = make_classification(n_samples=1000, n_features=20,
                           n_informative=2, n_redundant=10, random_state=42)

# Split the data into 80% training data and 20% validation (test) data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Define preprocessing and model pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', SVC())
])

# Define the parameter space for grid search
param_grid = {
    'classifier__C': [0.1,0.5, 1,5, 10],
    'classifier__kernel': ['linear', 'rbf'],
    'classifier__gamma': ['scale', 'auto']
}


In [4]:
# Initialize the grid search and fit it on the data
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print('Best parameters:', best_params)

# Score the model on the test data
score = grid_search.score(X_test, y_test)
print('Test score:', score)


Best parameters: {'classifier__C': 0.5, 'classifier__gamma': 'scale', 'classifier__kernel': 'rbf'}
Test score: 0.88


In [5]:
# Create a new SVM model with the best parameters
best_svm_model = SVC(
    C=best_params['classifier__C'], gamma=best_params['classifier__gamma'], kernel=best_params['classifier__kernel'])

# Fit the model on the training data
best_svm_model.fit(X_train, y_train)


SVC(C=0.5)

In [6]:

# Make predictions on the validation data
y_pred = best_svm_model.predict(X_test)

y_predGrid = best_svm_model.predict(X_test)

precisionGrid = precision_score(y_test, y_predGrid)
recallGrid = recall_score(y_test, y_predGrid)

print('Precision:', precisionGrid)
print('Recall:', recallGrid)


Precision: 0.8735632183908046
Recall: 0.8539325842696629


In [7]:
from sklearn.kernel_approximation import Nystroem
# Perform Nystroem approximation to approximate the multiple kernels
n_components = 15  # Number of components/kernels
nystroem = Nystroem(n_components=n_components)
X_train_approx = nystroem.fit_transform(X_train)
X_test_approx = nystroem.transform(X_test)

# Train an SVM classifier on the approximated data
svm = SVC()
svm.fit(X_train_approx, y_train)

# Predict the labels of the test set
y_pred = svm.predict(X_test_approx)

# Calculate precision and recall
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print('Precision:', precision)
print('Recall:', recall)


Precision: 0.8636363636363636
Recall: 0.8539325842696629


In [14]:
from shogun import RealFeatures, BinaryLabels, MulticlassLabels, KernelMulticlassMachine, GaussianKernel, PolyKernel

ImportError: cannot import name 'RealFeatures' from 'shogun' (c:\Users\Alireza Nasoodi\AppData\Local\Programs\Python\Python310\lib\site-packages\shogun\__init__.py)

In [11]:
# Convert data to Shogun format
features_train = RealFeatures(X_train.T)
labels_train = BinaryLabels(y_train)

features_val = RealFeatures(X_test.T)
labels_val = BinaryLabels(y_test)

# Create kernel objects
kernel_linear = GaussianKernel(features_train, features_train, 1.0)
kernel_rbf = GaussianKernel(features_train, features_train, 2.0)
kernel_poly = PolyKernel(features_train, features_train, 2.0, 2.0)

# Create the multi-kernel SVM model
machine = KernelMulticlassMachine()
machine.set_labels(labels_train)

# Set up the multi-kernel with different weights
machine.add_kernel(kernel_linear)
machine.add_kernel(kernel_rbf)
machine.add_kernel(kernel_poly)

# Train the model
machine.train()

# Make predictions on the validation data
predictions = machine.apply_multiclass(features_val).get_labels()

# Convert predictions to numpy array
predictions = np.array(predictions)

# Calculate precision and recall
precision = precision_score(y_val, predictions)
recall = recall_score(y_val, predictions)

print("Precision:", precision)
print("Recall:", recall)


NameError: name 'RealFeatures' is not defined